Download Jupyter Notebook File: [ipynb File](/notebooks/2023-07-08-false-claims-about-solar-potential.ipynb)

In [1]:
import handcalcs.render
import pandas as pd
import forallpeople as si
# try:
si.environment('energy_custom')
# kg m s A cd K mol

## [PV-magazine](https://www.pv-magazine.com/2023/03/23/assessing-the-area-intensity-of-pv/) "Assessing the area intensity of PV"

>Typically, developed countries such as the United States, Australia and Singapore consume about 10 MWh of electricity per person per annum. This will need to double to accommodate the electrification of transport, heating, and industry. Assuming 22% efficient panels and a DC capacity factor of 17% (averaged across rooftop solar and solar farms), we arrive at a figure of 13 kW per person occupying 60 m<sup>2</sup>.

They do not state _how_ they arrived at that figure. They missed out the figure of __capacity per unit panel area__ which is normally [150Wp/m²](http://www.solarelectricuk.com/datasheets/Glass-Model-STKP-72-310-Polycrystalline-Cell-Module.pdf). The convention is to use 'Wp' for watts, with the 'p' standing for 'peak'. Trying to replicate their calculations.


In [2]:
hours = 24 * si.hour
Wh = si.Wh
kWh = si.Wh * 1e3
MWh = si.Wh * 1e6
hr = si.hour
day = si.day
sec = si.s
year = si.year
W = si.W
kW = si.W * 1e3
TW = si.W * 1e12
TWh = si.Wh * 1e12
kg = si.kg
tonne = si.kg * 1000
DCPower = 13e3 * W
A = 60 * si.m**2
Efficiency = 0.22
LUE = 150 * W /si.m**2
C_f = 0.17

PVMagazinePrediciton = 13 *kW

In [3]:
%%render  2

ActualPower = A  * LUE * Efficiency  * C_f


<IPython.core.display.Latex object>

In [4]:
%%render 0
Error = PVMagazinePrediciton / ActualPower

<IPython.core.display.Latex object>


Their numbers, are wrong by a factor of 39. And even if it were right, it misses a number of confounding inefficiencies. Much is bound up into the capacity factor, and it has to account to everything.

Now we have seen the wrong way to do it, lets find a better way.

## [Joule  Volume 5, Issue 5:](https://doi.org/10.1016/j.joule.2021.03.005) 'Solar photovoltaics is ready to power a sustainable future'

This one was published in a scientific journal. PV magazine managed to milk this one, with a couple of separate articles all refering back to the source study:

- [pv magazine:"Solar may cover the world’s electricity demand with 0.3% of its land area"](https://www.pv-magazine.com/2023/02/01/solar-may-cover-the-worlds-electricity-demand-with-0-3-of-its-land-area/) 
- [pv magazine: "Solar still largely underestimated"](https://www.pv-magazine.com/2021/03/31/solar-still-largely-underestimated/)

One claim, which does not make it into the summary (there is no abstract), but which is rather striking is:

> "Supplying the current global electricity consumption with solar PV would imply covering 0.3% of the land area of the world"


In getting to that they claim, in ['supplemental notes 3'](https://www.cell.com/cms/10.1016/j.joule.2021.03.005/attachment/c2b79392-88f9-4231-833d-faf1a0a2a125/mmc1.pdf) an optimistic but feasible '[installed capacity](https://sebbeck.com/2023/06/13/solar-land-area#installed)', a.k.a. land use efficiency of 51W/m²:

>"Assuming 17% efficiency and that only 30% of the land is covered by solar panels in large-scale installations, the capacity density results in 51 W/m2, which is in agreement with values reported in [2013 NREL study called "Land-use requirements for solar power plants in the United States"](https://www.nrel.gov/docs/fy13osti/56290.pdf)

Although the number is in the bounds of the reasonable, it is technically wrong on two counts. Firstly, it isn't possible to conclude installed capacity from efficiency and land coverage factor of panels. Secondly, the reference states a range of capacity densities for PV panels from as low as 26 W/m² for 1MW farms with 2-axis movement panels up to 43W/m² for >20MW farms with fixed panels. Not 51W/m².





In [5]:
from pint import UnitRegistry, set_application_registry

u = UnitRegistry()
set_application_registry(u)
u.default_format = '~'
u.define("USD = [money]")  


In [6]:
%%render params
LUE = 51 * u.W / u.m**2


<IPython.core.display.Latex object>

I'll convert the obfuscating figure of 1370 kWh/kW into capacity factor by dividing by 24:

In [7]:
%%render params
C_f =  (1370 * u.kWh/u.kW/ (24 * 365.25 * u.h)).to_base_units()


<IPython.core.display.Latex object>

In [8]:
%%render
P_solar = LUE * C_f


<IPython.core.display.Latex object>

Now to get land requirements:

In [9]:
E = 27000 * u.TWh
year  = u.year

In [10]:
%%render  1

P = (E /year).to(u.TW)


<IPython.core.display.Latex object>

In [11]:
A_world = (13003e6 * u.ha).to(u.Mm**2)

In [12]:
%%render
A_solar = (P / P_solar).to(u.Mm**2)
LandPercent = 100 * A_solar / A_world #\%

<IPython.core.display.Latex object>

The literal claim is true. However, repeating this without any context is irresponsible and wrong in the technical sense. A grid reliant on a randomly variable, weather-dependant resource, is useless without energy storage. These are the only times energy storage is mentioned:

> For higher penetrations, short-term storage with high-efficiency, i.e., electric batteries, pumped hydro storage (PHS),99 and demand-side management contribute to energy arbitrage to ease the intraday balancing of solar PV-.100

> The short-term storage provided by EV batteries synergizes with PV, easing the daily balancing of solar generation,112 in particular when demand-side management and vehicle-to-grid capabilities are assumed for EVs.

> As solar becomes cheaper, it begins to compete with gas and coal for industrial heating through the use of high-efficiency electric furnaces. This can be coupled with low-cost on-site thermal storage in water, hot rocks, molten salts, molten silicon, and other materials to take advantage of low-cost daytime solar electricity.

Notice how no numbers are applied to establish energy storage requirements. Lets run the number just to remind ourselves. Lets pick a [random location](https://globalsolaratlas.info/detail?c=16.972741,-55.371094,3&s=28.921631,-96.679688&m=site), in Texas. I picked somewhere near the equator where the intra-day storage problem is lowest.

![](/notebooks/texas-solar.png)


Naturally the daily power-profiles reflect the presence of the sun...
![](/notebooks/texas-power.jpg)

Now, lets look at the electrical consumption profile of texas on a [random day](https://www.eia.gov/electricity/gridmonitor/dashboard/custom/pending). I've extracted the data [in this CSV](/notebooks/texas-power.csv)

![](/notebooks/day-vs-night.svg)

Almost half of consumption occurs during the night. So, using the ball park numbers...

In [13]:
%%render params 
E_nightly = E * 0.48


<IPython.core.display.Latex object>

The most authorative source I could find for the lithium content was a [LinkedIn](https://www.fedex.com/content/dam/fedex/us-united-states/services/LithiumBattery_JobAid.pdf) article:


In [14]:
%%render params
M_LithiumContent = 0.167* u.kg/u.kWh



<IPython.core.display.Latex object>

So the mass of lithium required to get us enough battery storage to make it through one medium length equatorial night is...

In [15]:
%%render
M_LithiumStorage = (E_nightly * M_LithiumContent).to(u.Gt)



<IPython.core.display.Latex object>

According to the [Canadian government](https://natural-resources.canada.ca/our-natural-resources/minerals-mining/minerals-metals-facts/lithium-facts/24009):

> More than half of the estimated global lithium resources are located in the salt flats of Bolivia, Chile and Argentina, much of that in Bolivia. It is estimated that there are 89 million tonnes of lithium resources globally.



[USGS estimates the 'reserves'](https://www.usgs.gov/centers/national-minerals-information-center/lithium-statistics-and-information) in PDF documents.  Some of the [the raw data is here.](https://www.sciencebase.gov/catalog/item/63d1a467d34e06fef150071d). I have had to manually compile it into the below table. The mine production reports for a given tend to grow year on year, presumably as more data comes in, so take the production column with a hint of salt.



In [16]:
df = pd.read_csv("lithium.csv")
df

,Year,Lithium Carbonate USD/Tonne,Reserves tonnes,Mine Production tonnes
0,2018,16000,16000000.0,95000
1,2019,11700,14000000.0,77000
2,2020,8400,17000000.0,82500
3,2021,12600,21000000.0,100000
4,2022,37000,22000000.0,130000
5,2023,?,26000000.0,?


USGS defines reserves as:

> Reserves.—That part of the reserve base which could be economically extracted or produced at the time of determination. The term reserves need not signify that extraction facilities are in place and operative. Reserves include only recoverable materials; thus, terms such as “extractable reserves” and “recoverable reserves” are redundant and are not a part of this classification system.

Therefore, the canadian estimate seems to include identified, speculative reserves.

It also notes that reserves grow.

>  For example, in 1970, identified and undiscovered world copper resources were estimated to contain 1.6 billion metric tons of copper, with reserves of about 280 million tons of copper. Since then, almost 560 million tons of copper have been produced worldwide, but world copper reserves in 2019 were estimated to be 870 million tons of copper, more than triple those of 1970, despite the depletion by mining of more than the original estimated reserves. 


Acknowledging that the reserves and price will all ways be changing, but using the canadian identified figure, the world is many times short of the lithium to make it through one night of electricity:

In [17]:
%%render params 0
M_LithiumGlobal = 89 * u.megatonne


<IPython.core.display.Latex object>

In [18]:
%%render params 0
Ratio = (M_LithiumStorage / M_LithiumGlobal).to_base_units() # Times more than global speculated reserves

<IPython.core.display.Latex object>

And using the price of Lithium carbonate for 2023...

In [19]:
%%render params 0
Cost_LithiumCarbonate = 37000 * u.USD / u.kg



<IPython.core.display.Latex object>

In [20]:
%%render params 2
LithiumProportion = 0.19 # of Lithium carbonate by mass
Cost_Lithium = Cost_LithiumCarbonate  / LithiumProportion



<IPython.core.display.Latex object>

In [21]:
%%render params 0
Cost_NightlyStorage = (M_LithiumStorage * Cost_Lithium).to(u.teraUSD)
GlobalGDP = 100 * u.teraUSD


<IPython.core.display.Latex object>

In [22]:
%%render params 0
Ratio = (Cost_NightlyStorage / GlobalGDP).to_base_units() # times GDP

<IPython.core.display.Latex object>

In terms of the global rate of lithium mining, it would also take an incredibly long time. Lithium production is rising as an incredible velocity. From 2022-23 appear to have seen more than 30% growth. But even at this rate, and assuming 100% of the lithium is used for batteries, its physically impossible when you consider that batteries need recycling every 10 years.

In [23]:
%%render  params 0
M_LithiumStorage
ProductionRate = 130_000 * u.tonnes / u.year


<IPython.core.display.Latex object>

In [24]:
%%render params 0
Time = (M_LithiumStorage / ProductionRate).to(u.year) # to mine the necessary lithium

<IPython.core.display.Latex object>

So neither in terms of budget, Lithium reserves or production rate is it feasible to produce the lithium required for even the most basic requirement of a single night's storage.